# Рекомендация тарифов

**Данные:** поведение клиентов, которые уже перешли на тарифы Smart и Ultra. 

Примечание: предобработка данных уже сделана.

**Описание данных:**
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Цель исследования:** построить модель с максимально большим значением *accuracy* для задачи классификации, которая выберет подходящий тариф (smart или ultra) для клиентов архивных тарифов. 

**Ход исследования:**
Данные о поведении клиентов находятся в файле users_behavior.csv. Качество данных приемлемо, сделана предобработка. Я разобью данные на 3 выборки: тренировочную, валидационную и тестовую и проверю 3 разных модели, рассчитаю метрику Accuracy, выберу лучшую модель. Также сделаю перебор мегапараметров. В конце лучшую модель проверю на тестовой выборке и посмотрю на адекватность.

**Исследование пройдёт в несколько этапов:**

- Импорт библиотек.
- Общая информация о данных.
- Разбивка данных на выборки.
- Исследование моделей.
- Проверка лучшей модели на тестовой выборке.
- Проверка модели на адекватность.
- Вывод.

## Импорт библиотек

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Общая информация о данных

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head())
display(df.info())
df.describe()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Промежуточный вывод:** с данными все в порядке, можно приступать к разбивке на выборки.

## Разбивка данных на выборки

Сначала я разобью данные на тренировочную и валидационную выборку, а потом валидационную разобью пополам. Так я получу 3 выборки с соотношением:
- тренировочная выборка 0.6
- валидационная выборка 0.2
- тестовая выборка 0.2

In [3]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

print(features_train.shape, features_valid.shape, features_test.shape) #проверка размера выборок
print(target_train.shape, target_valid.shape, target_test.shape)

(1928, 4) (643, 4) (643, 4)
(1928,) (643,) (643,)


## Исследование моделей

Целевой признак категориальный, это задача бинарной классификации.

### Классификатор дерева решений

Проверю метрику Accuracy с разными depth, criterion и splitter:

In [4]:
best_model = None
best_result = 0

for depth in range(1, 11):
    for crit in ['gini', 'entropy']:
        for spl in ['best', 'random']:
            model = DecisionTreeClassifier(random_state=12345, max_depth=depth, criterion=crit, splitter=spl)
            model.fit(features_train, target_train)
            predictions_valid = model.predict(features_valid)
            result = accuracy_score(target_valid, predictions_valid)
            if result > best_result:
                best_model = model
                best_result = result
                d = depth
                c = crit
                s = spl

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "; max_depth:", d, \
      "; criterion:", c, "; splitter:", s)

Accuracy наилучшей модели на валидационной выборке: 0.7884914463452566 ; max_depth: 7 ; criterion: entropy ; splitter: random


Эта модель получает максимум 0.789 по метрике Accuracy с максимальной глубиной дерева 7, критерием entropy и сплиттером random.

### Случайный лес

Проверю метрику Accuracy с разным количеством n_estimators и глубиной дерева:

In [5]:
best_model = None
best_result = 0

for est in range(1, 11):
    for md in range(1,11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=md)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            best_model = model
            best_result = result
            estim = est
            mdt = md

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "; n_estimators:", estim, "; max_depth:", mdt)

Accuracy наилучшей модели на валидационной выборке: 0.80248833592535 ; n_estimators: 8 ; max_depth: 8


Эта модель получает максимум 0.802 по метрике Accuracy с n_estimators 8 и глубиной 8.

### Логическая регрессия

Проверю метрику Accuracy с разным количеством max_iter и шагом 100:

In [6]:
best_model = None
best_result = 0

for max_it in range(100, 1101, 100):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=max_it)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        maxit = max_it

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "; max_iter:", maxit)

Accuracy наилучшей модели на валидационной выборке: 0.7107309486780715 ; max_iter: 100


Эта модель получает максимум 0.711 по метрике Accuracy с max_iter 100.

**Вывод по моделям:**

| Название модели              | Мегапараметры                                        | Accuracy           |
| :----------------------------|:-----------------------------------------------------| ------------------:|
| Классификатор дерева решений | max_depth: 7 ; criterion: entropy ; splitter: random | 0.7884914463452566 |
| Случайный лес                | n_estimators: 8 ; max_depth: 8                       | 0.80248833592535   |
| Логическая регрессия         | max_iter: 100                                        | 0.7107309486780715 |

Лучшая модель - случайный лес (RandomForestClassifier) с n_estimators: 8 ; max_depth: 8

## Проверка лучшей модели на тестовой выборке

Так как я выбрала модель RandomForestClassifier с n_estimators=8, max_depth=8, проверю ее на тестовой выборке и посчитаю Accuracy:

In [7]:
model = RandomForestClassifier(random_state=12345, n_estimators=8, max_depth=8)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)

print("Accuracy на тестовой выборке:", result)

Accuracy на тестовой выборке: 0.7962674961119751


Эта модель получает максимум 0.796 по метрике Accuracy.

**Вывод:** модель работает правильно и на тестовой выборке показала себя хорошо. Практически так же, как и на валидационной выборке.

## Вывод

Были созданы и исследованы на валидационной выборке 3 модели для решения задачи:

| Название модели              | Мегапараметры                                        | Accuracy           |
| :----------------------------|:-----------------------------------------------------| ------------------:|
| Классификатор дерева решений | max_depth: 7 ; criterion: entropy ; splitter: random | 0.7884914463452566 |
| Случайный лес                | n_estimators: 8 ; max_depth: 8                       | 0.80248833592535   |
| Логическая регрессия         | max_iter: 100                                        | 0.7107309486780715 |

    
Лучше всего себя проявила модель случайного леса (RandomForestClassifier).
Проверка на тестовой выборке подтвердила, что модель работает корректно, она получила 0.7962674961119751 по метрике Accuracy.